In [2]:
# %cd ..
%cd /home/janneke/Documents/Master/Machine_Learning_in_Practice/HMS/MLiP_group_10_task1_HMS/

/home/janneke/Documents/Master/Machine_Learning_in_Practice/HMS/MLiP_group_10_task1_HMS


In [3]:
from copy import deepcopy

import pandas as pd
import numpy as np
from logging import getLogger, basicConfig, INFO
from sklearn.model_selection import train_test_split
import lightgbm as lgb

from datasets.raw_data_loader import CustomRawDataset
from generics.configs import DataConfig
from utils.evaluation_utils import score_kl_divergence

In [4]:
basicConfig(level=INFO)
logger = getLogger('main')
config = DataConfig()

In [5]:
dataset = CustomRawDataset(config, subset_sample_count=100, mode="train", cache=True)
dataset.print_summary()

2024-02-15 16:01:03,524 - utils.general_utils - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_100_train.npz
INFO:utils.general_utils:Loading dataset from cache: ./data/cache/CustomRawDataset_100_train.npz


Dataset Summary:
Mode: train
Total Samples: 100
Unique Patients: 100
Unique EEGs: 100
Unique Spectrograms: 100
Label Distribution:
seizure_vote    145
lpd_vote         60
gpd_vote         46
lrda_vote        50
grda_vote        57
other_vote      258
dtype: int64

Vote Statistics:
        seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  other_vote
mean        1.450000  0.600000  0.460000   0.500000   0.570000    2.580000
median      0.000000  0.000000  0.000000   0.000000   0.000000    0.000000
var         2.775253  4.525253  4.149899   1.141414   1.378889   25.114747
Probabilities Loaded: 100
Features Loaded: 100

Configuration Summary:
+---------------------+--------------------------------------------------------------------------------------------------------------------------------------+
| Configuration       | Value                                                                                                                                |
+---------------------+------

In [6]:
x_train, x_test, y_train, y_test = train_test_split(dataset.features_per_sample, dataset.lbl_probabilities, test_size=0.33, random_state=42)
y0, y1, y2, y3, y4, y5 = zip(*y_train)

Let multiple separate models regress on one label each

In [7]:
y_pred = np.zeros(y_test.shape)


# Compare: https://github.com/microsoft/LightGBM/blob/ffb298626e131b9d213b34b3655846be441fdc12/src/objective/regression_objective.hpp#L127
def multi_mse(y_hat, dtrain): 
    """Based on the simplified MSE commonly used in GBM models"""
    y_true = y_train.values
    grad = y_hat.reshape(y_true.shape, order="F") - y_true
    hess = np.ones_like(y_true)
    return grad.flatten("F"), hess.flatten("F")

# We should choose a relatively small value of min_data_in_leaf in this task
# Otherwise, we'll get:
# [LightGBM] [Warning] No further splits with positive gain, best gain: -inf
params = {
    "num_class": 6,
    "random_seed": 42,
    "verbose": 1,
    "num_boost_round": 100,
    "min_data_in_leaf": 5
}

# Please note, that we don't pass y_train as label argument of 
# the Dataset class object, because LightGBM doesn't accept 
# multiple target columns and raises the following error:
# (ValueError: DataFrame for label cannot have multiple columns).
dtrain = lgb.Dataset(x_train)

lgb_model = lgb.train(
    train_set = dtrain,
    params    = params,
    feval      = multi_mse
)

y_pred = lgb_model.predict(x_test)

/home/janneke/anaconda3/envs/MLiP/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[LightGBM] [Fatal] Number of classes must be 1 for non-multiclass training


LightGBMError: Number of classes must be 1 for non-multiclass training

In [ ]:
y_pred[y_pred < 0] = 0
y_pred_probabilities = y_pred / np.sum(y_pred, axis=1)[:,None]

In [ ]:
submission = pd.DataFrame(y_pred_probabilities, columns=dataset.label_cols)
solution = pd.DataFrame(y_test, columns=dataset.label_cols)

# Need to be added for score calculation, but are removed at the start of the function
submission.insert(0, "id", range(len(submission)))
solution.insert(0, "id", range(len(solution)))

score_kl_divergence(solution=deepcopy(solution), submission=deepcopy(submission), row_id_column_name="id")

7.462399807301511

In [ ]:
# Random
random_pred = np.random.rand(33, 6)
random_pred[random_pred < 0] = 0
random_pred_probabilities = random_pred / np.sum(random_pred, axis=1)[:,None]
random_submission = pd.DataFrame(random_pred_probabilities, columns=dataset.label_cols)

random_submission.insert(0, "id", range(len(random_submission)))

score_kl_divergence(solution=deepcopy(solution), submission=deepcopy(random_submission), row_id_column_name="id")

1.7544822016819417